事前に、適当なEC2インスタンスを立ち上げます。のちほど使用します。
- リージョン＝東京
- AMI＝Amazon Linux 2
- インスタンスタイプ＝任意（t2.microなど）
- VPC＝デフォルトVPC（default-vpc）
- セキュリティグループ＝デフォルト（SSH接続を許可）

In [ ]:
import boto3
ec2 = boto3.resource('ec2')

In [ ]:
# デフォルトVPCを取得
default_vpc = [vpc for vpc in ec2.vpcs.all() if vpc.is_default][0]

default_vpc

In [ ]:
# デフォルトVPCにキャッシュクラスター用セキュリティグループを作成

elasticache_sg = default_vpc.create_security_group(
    GroupName='elasticache_sg', 
    Description='elasticache_sg')

elasticache_sg

In [ ]:
# セキュリティグループで、TCPポート11211の接続を許可
elasticache_sg.authorize_ingress(
    IpProtocol='tcp', 
    FromPort=11211, 
    ToPort=11211, 
    CidrIp='0.0.0.0/0')

In [ ]:
import boto3
ec = boto3.client('elasticache')


In [ ]:
# キャッシュクラスターを作成

response = ec.create_cache_cluster(
    CacheClusterId='test3', 
    Engine='memcached', 
    CacheNodeType='cache.t2.micro', 
    NumCacheNodes=1,
    SecurityGroupIds=[elasticache_sg.id])

response

In [ ]:
# すべてのキャッシュクラスターを表示
ec.describe_cache_clusters()

In [ ]:
# 状態を確認。「creating」から「available」に変わったら、次に進めます。
ec.describe_cache_clusters()['CacheClusters'][0]['CacheClusterStatus']

In [ ]:
# キャッシュクラスターのエンドポイントを確認

endpoint = ec.describe_cache_clusters()['CacheClusters'][0]['ConfigurationEndpoint']

address = '%s:%s' % (endpoint['Address'], endpoint['Port'])
print('ElastiCacheのアドレス:', address)

ElastiCacheには、デフォルトでは、VPC外からは接続ができません。
https://docs.aws.amazon.com/ja_jp/AmazonElastiCache/latest/mem-ug/accessing-elasticache.html

したがって、冒頭で準備したEC2インスタンスにSSH接続し、下記を実行します。
```
sudo yum -y install python3
pip3 install python-memcached --user
python3
```

以降、上記で実行したpython内で実行（>>>以下に貼り付け）

In [ ]:
# クライアントの作成
import memcache

address = 'test3.xhpium.cfg.apne1.cache.amazonaws.com:11211' # 上記で出力されたElastiCacheのアドレスをセット。
mc = memcache.Client([address], debug=1)

In [ ]:
# キャッシュへの書き込み
mc.set('name', 'taro')

In [ ]:
# キャッシュからの読み込み
mc.get('name')